In [1]:
import pandas as pd
import numpy as np

ks2cols = ["URN", "SCHNAME", "PTKS1GROUP_M", "PTKS1GROUP_H", "PTREAD_EXP" ,"PTREAD_HIGH", "PTGPS_EXP", "PTGPS_HIGH",
          "PTWRITTA_EXP", "PTWRITTA_HIGH", "PTMAT_EXP", "PTMAT_HIGH", "PTRWM_EXP", "PTRWM_HIGH"]
ks4cols = ["URN", "SCHNAME", "ATT8SCR", "P8MEA"]
ks2dtypes = {"URN":"int32", "SCHNAME":"str", "PTKS1GROUP_M":"float64", "PTKS1GROUP_H":"float64", "PTREAD_EXP":"float64",
             "PTREAD_HIGH":"float64", "PTGPS_EXP":"float64", "PTGPS_HIGH":"float64", "PTWRITTA_EXP":"float64", 
             "PTWRITTA_HIGH":"float64", "PTMAT_EXP":"float64", "PTMAT_HIGH":"float64", "PTRWM_EXP":"float64", 
             "PTRWM_HIGH":"float64"}
ks4dtypes = {"URN":"int32", "SCHNAME":"str", "ATT8SCR":"float64", "P8MEA":"float64"}

# Import Educational Data spreadsheets
ks22017 = pd.read_csv('../../Mini-project-London/data/data-education-raw/england_ks2final17.csv', usecols=ks2cols)
ks42017 = pd.read_csv('../../Mini-project-London/data/data-education-raw/england_ks4final17.csv', usecols=ks4cols)
ks22018 = pd.read_csv('../../Mini-project-London/data/data-education-raw/england_ks2final18.csv', usecols=ks2cols)
ks42018 = pd.read_csv('../../Mini-project-London/data/data-education-raw/england_ks4final18.csv', usecols=ks4cols)
ks22019 = pd.read_csv('../../Mini-project-London/data/data-education-raw/england_ks2final19.csv', usecols=ks2cols, 
                      nrows = 16355)
ks42019 = pd.read_csv('../../Mini-project-London/data/data-education-raw/england_ks4final19.csv', usecols=ks4cols)

# Import schools exceeding limits spreadsheet
schools_exced = pd.read_excel('../../Mini-project-London/data/data-pollution/Schools_exceeding_LAEI_2016_2-modified.xlsx'
                              , sheet_name='EdubaseAllEstablishmentsLondon')

# Concatenate data arrays
ks2 = pd.concat([ks22017, ks22018, ks22019])
ks4 = pd.concat([ks42017, ks42018, ks42019])

# Data cleaning - remove unwanted values
for column in ks2:
    ks2 = ks2.loc[~((ks2[column].isin([" ", "SUPP", "NA", "NP", "NE", "LOWCOV"])) | (ks2[column].isna()))]
    ks2[column] = ks2[column].map(lambda s: float(s.rstrip("%")) / 100.0 if str(s)[-1]=="%" else s)

for column in ks4:
    ks4 = ks4.loc[~((ks4[column].isin([" ", "SUPP", "NA", "NP", "NE", "LOWCOV"])) | (ks4[column].isna()))]
    ks4[column] = ks4[column].map(lambda s: float(s.rstrip("%")) / 100.0 if str(s)[-1]=="%" else s)    

# Convert to correct datatypes
ks2 = ks2.astype(ks2dtypes)
ks4 = ks4.astype(ks4dtypes)

# Find average pass rate for KS2 subjects
ks2["ks2 % expected"] = (ks2["PTREAD_EXP"] + ks2["PTGPS_EXP"] + ks2["PTWRITTA_EXP"] + ks2["PTMAT_EXP"])/4
ks2["ks2 % high"] = (ks2["PTREAD_HIGH"] + ks2["PTGPS_HIGH"] + ks2["PTWRITTA_HIGH"] + ks2["PTMAT_HIGH"])/4
ks2 = ks2.drop(["PTREAD_EXP" ,"PTREAD_HIGH", "PTGPS_EXP", "PTGPS_HIGH", "PTWRITTA_EXP", "PTWRITTA_HIGH", "PTMAT_EXP", 
                "PTMAT_HIGH", "PTRWM_EXP", "PTRWM_HIGH"], axis =1)

# Group by URN and average other columns
ks2 = ks2.groupby(["URN"]).mean()
ks4 = ks4.groupby(["URN"]).mean()

# Inner join on URN
schoolexced_ks2 = pd.merge(schools_exced, ks2, on="URN")
schoolexced_ks4 = pd.merge(schools_exced, ks4, on="URN")

correlation_ks2 = schoolexced_ks2.corr()
correlation_ks4 = schoolexced_ks4.corr()
correlation_ks2




C:\Users\fergu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,URN,NO2 Average 2013 (µg/m3),NO2 Average 2016 (µg/m3),PTKS1GROUP_M,PTKS1GROUP_H,ks2 % expected,ks2 % high
URN,1.000000,-0.226274,-0.240131,-0.048736,-0.040342,-0.083079,-0.078557
NO2 Average 2013 (µg/m3),-0.226274,1.000000,0.979296,0.140673,-0.121833,0.048174,0.020160
NO2 Average 2016 (µg/m3),-0.240131,0.979296,1.000000,0.154810,-0.138585,0.051686,0.019017
PTKS1GROUP_M,-0.048736,0.140673,0.154810,1.000000,-0.071544,0.600213,0.186206
PTKS1GROUP_H,-0.040342,-0.121833,-0.138585,-0.071544,1.000000,0.652858,0.696410
ks2 % expected,-0.083079,0.048174,0.051686,0.600213,0.652858,1.000000,0.796885
ks2 % high,-0.078557,0.020160,0.019017,0.186206,0.696410,0.796885,1.000000


In [2]:
correlation_ks4

,URN,NO2 Average 2013 (µg/m3),NO2 Average 2016 (µg/m3),ATT8SCR,P8MEA
URN,1.000000,-0.134320,-0.159847,0.166638,0.131079
NO2 Average 2013 (µg/m3),-0.134320,1.000000,0.978811,0.002282,0.061228
NO2 Average 2016 (µg/m3),-0.159847,0.978811,1.000000,-0.002538,0.067300
ATT8SCR,0.166638,0.002282,-0.002538,1.000000,0.907456
P8MEA,0.131079,0.061228,0.067300,0.907456,1.000000
